# CPU-bound tasks

In [2]:
import time
import threading

def cpu_bound_job(job_id, num):
    start_job = time.time()    
    print(f"CPU-bound sub-job {job_id} started.")
    sum = 0
    # Simulated CPU-heavy arithmetic calculation.
    for i in range(num):
        sum += i
    duration = time.time() - start_job
    print(f"CPU-bound sub-job {job_id} finished in {duration:.2f} seconds.", end='\n')

def run_with_threads(n_jobs, num):
    threads = []
    for _id in range(n_jobs):
        # `args` is a tuple specifying the positional arguments for the
        # target function, which will be run in an independent thread.
        thread = threading.Thread(target=cpu_bound_job, args=(_id, num))
        threads.append(thread)
        thread.start()

    for thread in threads:
        # With `join`, we wait until the thread terminates, either normally
        # or through an unhandled exception.
        thread.join()

start_one_thread = time.time()
# Run with a single thread, with a big number.
run_with_threads(n_jobs=1, num=3*10**8)
duration = time.time() - start_one_thread
print(f"CPU-bound job finished in {duration:.2f} seconds with a single thread.")

start_three_threads = time.time()
# Run with three threads with a smaller number. The total number of three threads
# adds up to the one of a single thread so the result is comparable.
run_with_threads(n_jobs=3, num=10**8)
duration = time.time() - start_three_threads
print(f"CPU-bound job finished in {duration:.2f} seconds with three threads.")

CPU-bound sub-job 0 started.
CPU-bound sub-job 0 finished in 10.27 seconds.
CPU-bound job finished in 10.27 seconds with a single thread.
CPU-bound sub-job 0 started.
CPU-bound sub-job 0 finished in 3.41 seconds.
CPU-bound sub-job 1 started.
CPU-bound sub-job 1 finished in 3.45 seconds.
CPU-bound sub-job 2 started.
CPU-bound sub-job 2 finished in 3.47 seconds.
CPU-bound job finished in 10.34 seconds with three threads.


[How to write concurrent Python code with multithreading | by Lynn Kwong | Level Up Coding](https://levelup.gitconnected.com/how-to-write-concurrent-python-code-with-multithreading-b24dec228c43)
> When we split the big number into a smaller one and try to do the calculation with three threads, the calculation is not faster than that with a single thread. It means that multi-threading is not applicable for CPU-bound tasks and you should not waste your time trying to speed up CPU-bound applications with multi-threading. Instead, you should try to use the `multiprocessing` package.

# IO-bound tasks

In [3]:
import time
import threading

def io_bound_job(job_id, num_requests):
    start_job = time.time()
    print(f"IO-bound sub-job {job_id} started.")
    for _ in range(num_requests):
        # IO-bound jobs spend most of the time waiting for responses.
        time.sleep(3)
    duration = time.time() - start_job
    print(f"IO-bound sub-job {job_id} finished in {duration:.2f} seconds.")

def run_with_threads(n_jobs, num_requests_per_job):
    threads = []
    for _id in range(n_jobs):
        thread = threading.Thread(target=io_bound_job, args=(_id, num_requests_per_job))
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()

start_one_thread = time.time()
# Run with a single thread:
run_with_threads(n_jobs=1, num_requests_per_job=3)
duration = time.time() - start_one_thread
print(f"IO-bound job finished in {duration:.2f} seconds with a single thread.")

start_three_threads = time.time()
# Run with three threads:
run_with_threads(n_jobs=3, num_requests_per_job=1)
duration = time.time() - start_three_threads
print(f"IO-bound job finished in {duration:.2f} seconds with three threads.")

IO-bound sub-job 0 started.
IO-bound sub-job 0 finished in 9.03 seconds.
IO-bound job finished in 9.03 seconds with a single thread.
IO-bound sub-job 0 started.
IO-bound sub-job 1 started.
IO-bound sub-job 2 started.
IO-bound sub-job 0 finished in 3.01 seconds.
IO-bound sub-job 1 finished in 3.03 seconds.IO-bound sub-job 2 finished in 3.03 seconds.

IO-bound job finished in 3.03 seconds with three threads.


# Parallel simulation prototype

In [1]:
import os
import time
import threading
import subprocess
import paramiko

def remote_simulate(host="158.132.134.38", user="knpob", local_dat_folder="D:\\knpob\\20230613-FE cluster\\output\\e2x1", remote_dat_folder="D:\\knpob\\e2x1", dat_file="e2x1.dat", wait_time=1):
    start_job = time.time()

    # setup ssh tunnel
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.load_system_host_keys()
    ssh.connect(hostname=host, username=user)

    # transfer dat file
    try:
        ssh.exec_command('mkdir {}'.format(remote_dat_folder))
        subprocess.check_output(['scp', dat_file, '{}@{}:{}'.format(user, host, remote_dat_folder)], shell=False, cwd=local_dat_folder)
        print("\n{}@{}: dat file received {} -> {}".format(user, host, os.path.join(local_dat_folder, dat_file), remote_dat_folder))
    except:
        print("\n{}@{}: dat file transfer failed".format(user, host))
        
    # launch simulation task
    # [ISSUE] wait for simulation completes
    try:
        print(time.time())
        ssh.exec_command('cd {}; & "C:\\Program Files\\MSC.Software\\Marc\\2019.0.0\\marc2019\\tools\\run_marc.bat" -jid {} -back no -nps 4 -nts 3 -nte 3 -nsolver 6'.format(remote_dat_folder, dat_file))
        print("\n{}@{}: simulation completed".format(user, host))
        print(time.time())
    except:
        print("\n{}@{}: simulation failed".format(user, host))

    # simulation results feedback
    # p.s. in production, .t19 should be replaced with .t16
    # [ISSUE] dir permission
    try:
        subprocess.check_output(['scp', 
            '{}@{}:{}'.format(user, host, os.path.join(remote_dat_folder, dat_file.replace('.dat', '.t19'))),
            os.path.join(local_dat_folder, str(int(time.time())), dat_file.replace('.dat', '.t19'))],
            shell=False)
        subprocess.check_output(['scp', '{}@{}:{}'.format(user, host, 
            os.path.join(remote_dat_folder, dat_file.replace('.dat', '.sts'))), 
            os.path.join(local_dat_folder, str(int(time.time())), dat_file.replace('.dat', '.sts'))],
            shell=False)
        print("\n{}@{}: simulation results feed-backwarded".format(user, host))
    except:
        print("\n{}@{}: simulation results feed-backward failed".format(user, host))

    duration = time.time() - start_job
    print("\n{}@{}: simulation task completed in {:.2f}s.".format(user, host, duration))

def run_with_threads(n_jobs):
    print("="*100)
    start_time = time.time()
    threads = []

    for id in range(n_jobs):
        thread = threading.Thread(target=remote_simulate, kwargs=
            {
                'remote_dat_folder': "{}_{}".format("D:\\knpob\\e2x1", id),
            })
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()
    
    duration = time.time() - start_time
    print("-"*100)
    print("simulation tasks completed in {:.2f}s with {} thread.".format(duration, n_jobs))

run_with_threads(n_jobs=1)

c:\ProgramData\anaconda3\envs\marc\lib\site-packages\paramiko\transport.py:32: CryptographyDeprecationWarning: Python 3.6 is no longer supported by the Python core team. Therefore, support for it is deprecated in cryptography. The next release of cryptography will remove support for Python 3.6.
  from cryptography.hazmat.backends import default_backend



knpob@158.132.134.38: dat file received D:\knpob\20230613-FE cluster\output\e2x1\e2x1.dat -> D:\knpob\e2x1_0
1687165442.487509

knpob@158.132.134.38: simulation completed
1687165442.5030267
D:\knpob\20230613-FE cluster\output\e2x1\1687165443\e2x1.t19

knpob@158.132.134.38: simulation results feed-backward failed

knpob@158.132.134.38: simulation task completed in 2.26s.
----------------------------------------------------------------------------------------------------
simulation tasks completed in 2.26s with 1 thread.
